In [1]:
%reset -sf

In [94]:
# LOAD THIS BEFORE START, run rmdl

#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

# import numpy as np
import scipy
import matplotlib.pyplot as plt

abc = "abcdefghijklmnopqrstuvwxyz"
abc_map = {c:i for i,c in enumerate(abc)}
M9 = 10**9 + 7  # 998244353
yes, no = "YES", "NO"
d4 = [(1,0),(0,1),(-1,0),(0,-1)]
d8 = [
    (-1,-1),(-1,0),(-1,1),
    (0,-1),(0,0),(0,1),
    (1,-1),(1,0),(1,1)]
d6 = [(2,0),(1,1),(-1,1),(-2,0),(-1,-1),(1,-1)]  # hexagonal layout
MAXINT = sys.maxsize

In [104]:
with open("/Users/htong/Downloads/input.txt") as f:
    srr = f.read().strip().split("\n")

In [96]:
srr = """
19, 13, 30 @ -2,  1, -2
18, 19, 22 @ -1, -1, -2
20, 25, 34 @ -2, -2, -4
12, 31, 28 @ -1, -2, -1
20, 19, 15 @  1, -5, -3
""".strip().split("\n")

In [105]:
xrr = []
yrr = []
zrr = []
vxrr = []
vyrr = []
vzrr = []

In [106]:
def get_intersection(
    # Define the points for the first line
    x1_line1, y1_line1, #= 1, 2
    x2_line1, y2_line1, #= 4, 6

    # Define the points for the second line
    x1_line2, y1_line2, #= 2, 4
    x2_line2, y2_line2, #= 6, 3
):

    def calculate_slope_intercept(x1, y1, x2, y2):
        slope = (y2 - y1) / (x2 - x1)
        intercept = y1 - slope * x1
        return slope, intercept

    def find_intersection(slope1, intercept1, slope2, intercept2):
        if slope1 == slope2:
#             print("The lines are parallel and do not intersect.")
            return None
        x_intersect = (intercept2 - intercept1) / (slope1 - slope2)
        y_intersect = slope1 * x_intersect + intercept1
        return x_intersect, y_intersect

#     # Define the points for the first line
#     x1_line1, y1_line1 = 1, 2
#     x2_line1, y2_line1 = 4, 6

#     # Define the points for the second line
#     x1_line2, y1_line2 = 2, 4
#     x2_line2, y2_line2 = 6, 3

    # Calculate the slope and intercept for each line
    slope1, intercept1 = calculate_slope_intercept(x1_line1, y1_line1, x2_line1, y2_line1)
    slope2, intercept2 = calculate_slope_intercept(x1_line2, y1_line2, x2_line2, y2_line2)
    
#     print(slope1, intercept1)
#     print(slope2, intercept2)

    # Find the intersection point
    intersection = find_intersection(slope1, intercept1, slope2, intercept2)

#     if intersection:
#         print(f"The intersection point of the two lines is: {intersection}")
        
    return intersection

In [107]:
for row in srr:
    arr = row.replace(",", "").split()
    xrr.append(int(arr[0]))
    yrr.append(int(arr[1]))
    zrr.append(int(arr[2]))
    vxrr.append(int(arr[-3]))
    vyrr.append(int(arr[-2]))
    vzrr.append(int(arr[-1]))

In [108]:
# xrr, yrr

In [109]:
# In this example, look for intersections that happen with an X and Y position each at least 7 and at most 27

In [110]:
# 200000000000000 and at most 400000000000000. Disregard the Z axis entirely.

In [118]:
val_x = []
val_y = []

In [119]:
res = 0
for i,(x,y,z,vx,vy,vz) in enumerate(zip(xrr, yrr, zrr, vxrr, vyrr, vzrr)):
    for j,(x2,y2,z2,vx2,vy2,vz2) in enumerate(zip(xrr, yrr, zrr, vxrr, vyrr, vzrr)):
        if i >= j:
            continue
        intersection = get_intersection(
            x, y,
            x+vx, y+vy,
            x2, y2,
            x2+vx2, y2+vy2,
        )
        if intersection is None:
            continue
        ix, iy = intersection
        t1 = (ix - x) / vx
        t2 = (ix - x2) / vx2
        
        val_x.append(ix)
        val_y.append(iy)
        
        if t1 == 0:
            raise
        
#         if 7 <= ix <= 27 and 7 <= iy <= 27:
        if 200000000000000 <= ix <= 400000000000000 and 200000000000000 <= iy <= 400000000000000:
            if t1 >= 0 and t2 >= 0:
                res += 1

In [122]:
# plt.scatter(val_x, val_y)

In [121]:
res

16502

In [64]:
xrra

[19, 18, 20, 12, 20]

In [25]:
""

''